In [23]:
# This script loads fraoms from girder, loads tracks from girder, 
# crops out a small image centered on the nuclei for 5 frames before and after the crossing.
# It has to deal with bad tracks.

from __future__ import division, print_function
from ipywidgets import interact, interactive, fixed, interact_manual    
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy as sp
from scipy import ndimage
from IPython.core.debugger import set_trace
import cv2
import sys
from pprint import pprint
from random import randint
from nuclei.foci import *
import nuclei.girder as g
import nuclei.girder.time_lapse_data as reader

plt.rcParams['image.cmap'] = 'gist_earth'
matplotlib.rcParams['figure.figsize'] = (11.0, 11.0)




In [30]:

# Hard coded to folder ids
# TODO: Iterate over series in a folder.
# /Gwenda/LawDNADamageFociData
# 5aaf02831fbb9006233ae6a2  073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54
#    series000: 5aaf02871fbb9006233ae6a3
#    ...
# 5a95b8361fbb904fdd27d09a  080217 BT549 NLS GFP 53BP1 mcherry_2017_08_02__17_46_03
# 5aaec5d21fbb9006233ad700  080817 BT549 NLS GFP 53BP1 mcherry_2017_08_08__17_27_10

# this function is defined in foci.py
# 073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54/series000
# 92 time steps in this series (pattern "time%04d"%idx )
# three shannels are read into a single image (BGR)
# Looks like the brightfield channel is the first (0)
source = reader.time_lapse_data()
# 073117 BT549_Sv40_Bt549_SV40 NLS GFP 53BP1 mcherry_2017_07_31__18_31_54series000
source.load('5ada43f41fbb9005ff7260eb')
num_series = source.get_number_of_series()

#parameters
series_idx = 0
before_time = 82
color =  (106, 204, 30)
row = 1
gap = 14





num_time_steps = source.get_series_length(series_idx)
series_folder = source.get_series_folder(series_idx)

after_time = before_time + 10
# load the image objects from girder for the two frames (before and after).
before_im, before_item_obj = source.get_image(series_idx, before_time)
after_im, after_item_obj = source.get_image(series_idx, after_time)

print("before %d %d %s"%(series_idx, before_time, before_item_obj['_id']))


# Load the tracks from girder from the two frames.
annotation = g.Annotation("tracks")
annotation.LoadFromItem(before_item_obj['_id'])
before_tracks = annotation.annot_obj['elements']
annotation.LoadFromItem(after_item_obj['_id'])
after_tracks = annotation.annot_obj['elements']

# convert the color to hex format.
hex_digits = "0123456789abcdef"
r1 = hex_digits[int(color[0]/16)]
r2 = hex_digits[color[0]%16]
g1 = hex_digits[int(color[1]/16)]
g2 = hex_digits[color[1]%16]
b1 = hex_digits[int(color[2]/16)]
b2 = hex_digits[color[2]%16]
hex_color = "#"+r1+r2+g1+g2+b1+b2
print(hex_color)


def find_track_by_color(tracks, hex_color):
    # shortest because mismerged tracks in after will be long
    best_track = None
    for e in tracks:
        if e['type'] == 'polyline' and e['lineColor'] == hex_color:
            if not best_track or len(best_track) < len(e['points']):
                best_track = e['points']
    return best_track

# find the track for this cell in the "before" frame.
before_track = find_track_by_color(before_tracks, hex_color)
after_track = find_track_by_color(after_tracks, hex_color)

print([t['lineColor'] for t in before_tracks])
print([t['lineColor'] for t in after_tracks])

print("%d, %d"%(len(before_track), len(after_track)))



before 0 82 5ada44be1fbb9005ff7261e4
#6acc1e
['#a8697d', '#abcf1a', '#3cc172', '#bac13b', '#1e68c8', '#ca8109', '#4ebc19', '#f35b52', '#6f3b69', '#ba05e4', '#45c336', '#55abfd', '#bc1b51', '#c43d2a', '#b3a665', '#d6ed05', '#fdf026', '#a95d6a', '#d5557e', '#166bbf', '#8f8819', '#417f08', '#5d7c02', '#8ff4e8', '#8be9f0', '#e0f6ba', '#0dec48', '#0cd65b', '#def113', '#a31a71', '#ae25be', '#f941fe', '#5900c0', '#480311', '#5896e0', '#46f17e', '#45beb1', '#5e98d6', '#1daec8', '#5b1d92', '#ad29a4', '#9970a7', '#8c0fa2', '#f47ad7', '#8225ae', '#cb168f', '#1549d3', '#aaba6f', '#cc05a2', '#346995', '#e34e56', '#385463', '#df60b6', '#1bcff1', '#6fbd72', '#73c1e1', '#bcc440', '#36b41b', '#dc6f35', '#34ee60', '#d38a7d', '#1e7a67', '#0d350e', '#fa2925', '#65cdb9', '#db52cc', '#b5ec9f', '#ffd883', '#887850', '#58a408', '#8d13aa', '#ffc485', '#8783ac', '#7f00f7', '#394ff1', '#956b28', '#ae730b', '#318c30', '#960bfb', '#622c6b', '#8f38dc', '#b4ca10', '#f7af53', '#ce7883', '#09c5a3', '#eb0326', '#1a8e60

TypeError: object of type 'NoneType' has no len()

In [31]:
# Get the images for these cells.
# TODO: Put the time value in the z chanell.
# Assume the last track point is from the image time step.

# these will not be in the right order.

before_images = []
after_images = []
cross_track = []

for i in range(10):
    # Assume the last track point is from the image time step.
    pt_idx = len(after_track) - i - 1
    if pt_idx >= 0:
        x,y,z = after_track[pt_idx]
        z = after_time - i
        cross_track.append((x,y,z))
        im, item_obj = source.get_image(series_idx, z)
        im = im[y-64:y+64, x-64:x+64, ...]
        after_images.append(im)

for i in range(10):
    # Assume the last track point is from the image time step.
    pt_idx = len(before_track) - i - 1
    if pt_idx >= 0:
        x,y,z = before_track[pt_idx]
        z = before_time - i
        cross_track.append((x,y,z))
        im, item_obj = source.get_image(series_idx, before_time-i)
        im = im[y-64:y+64, x-64:x+64, ...]
        before_images.append(im)
        #print((x,y))

before_images.reverse()
after_images.reverse()
cross_track.reverse()
        
                
fig, ax = plt.subplots(4,5, sharey=True, figsize=(12,10))
# We need this coplexity because the arrays might not have 10 images.
# It still does not work, but it does not matter. These are for debugging.
count = 0
for x in range(2):
    for y in range(5):
        if count < len(before_images):
            ax[x][y].imshow(before_images[count], aspect="auto")
        count += 1

count = 0
for x in range(2,4):
    for y in range(5):
        if count < len(after_images):
            ax[x][y].imshow(after_images[count], aspect="auto")
        count += 1

print(cross_track)




TypeError: object of type 'NoneType' has no len()

In [ ]:
g.upload_images(before_images + after_images, 'cross_%d_%d_%d'%(before_time, row, gap), series_folder['_id'])